## Kernel limit for a Random Fourier Feature model

The formula for the deterministic equivalent $R_{n,p}$ can be applied for every value of $p$. In addition, when $p \to \infty$, the random feature model reduces to a kernel machine where:
$$
K(x,x') = \mathbb{E}_{w \in \mu(w)}[\varphi(x, w) \varphi(x', w)]
$$
As shown in [Rahimi and Recht](https://people.eecs.berkeley.edu/~brecht/papers/07.rah.rec.nips.pdf), if we let $\phi(x,w) = \cos(\langle w,x \rangle)$ and $w \in \mathcal{N}(0, \gamma\mathbf{1}_d)$, then the liming kernel is the usual RBF kernel $K(x,x') =\exp(-\gamma\frac{||x-x'||^2}{2})$.

In this notebook, we aim at verifying whether the predictions for the deterministic equivalent in the kernel limit $p \to \infty$ are in agreement with the risk induced by a RBF kernel ridge regression problem. In particular, we will proceed as shown in Appendix C.3 Empirical Diagonalization:
1) We sample $N$ points $\vec x \in \mathbb{R}^d$ from $\mathcal{N}(0, \mathbf{1}_d)$
2) We sample $P$ points $\vec w \in \mathbb{R}^p$ from $\mathcal{N}(0, \gamma \mathbf{1}_d)$
3) We can now approximate the PDF over $\vec w$ and write $K(x,x') \approx \sum_j^P \varphi(\vec x,\vec w_j) \varphi(\vec x',\vec w_j)$ where $\varphi(\vec x, \vec w) = \frac{1}{\sqrt{p}}\cos(\langle \vec x, \vec w \rangle)$
4) Starting from the $N$ points $\vec x_i$, we'll build the $N\times N$ matrix $\tilde K$ where $\tilde K_{ij} = \frac{1}{N} K(\vec x_i, \vec x_j)$
5) The eigenvalues $\tilde \xi_k^2$ approximate the true eigenvalues $\xi_k^2$ of $\Sigma$ for large enough $N$
6) We obtain the decomposition of $f_\star(\vec x)$ with $\beta_{\star,k} \approx \tilde \beta_k = \frac{1}{N} \vec y^T \tilde \psi_k$

In [52]:
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
plt.style.use('science')

In [57]:
# Perform the empirical diagonalization of the kernel matrix...
def empiricalDiagonalization(target_function, gamma = 0.5):
    N = 5000   # as large as possible
    P = 5000 
    d = 1000

    X = np.random.randn(N, d)
    W = np.random.randn(d, P) * np.sqrt(2 * gamma)
    b = np.random.rand(1, P) * 2 * np.pi

    Projection = X @ W 
    Phi = np.sqrt(2.0 / P) * np.cos(Projection + b)
    Ka = Phi @ Phi.T 

    vals, vecs = np.linalg.eigh(Ka)
    vals = vals / N
    vals = vals[::-1]
    vecs = vecs[:, ::-1]

    y = target_function(X) 
    betas = (1 / np.sqrt(N)) * (vecs.T @ y)
    return np.array(vals), betas

# Solver for nu_K in the kernel limit
def solveNu1KernelLimit(n, lambd, Sigma, tolerance=1e-10, maxIter=1000):
    nu = lambd 
    
    for it in range(maxIter):
        df = np.sum(Sigma / (Sigma + nu))
        nu_new = lambd + (nu / n) * df
        
        if np.abs(nu_new - nu) < tolerance:
            return nu_new
        nu = nu_new
        
    return nu

# Compute the deterministic equivalent of the variance and bias (same as in previous notebook, but specialized for kernel limit)
def computeDeterministicEquivalent(n : int, lambd : float, noise : float, Sigma : np.array, beta : np.array):
    nu1 = solveNu1KernelLimit(n, lambd, Sigma)

    BiasNum = nu1**2 * np.sum(beta**2 / (Sigma + nu1)**2 )
    BiasDen = 1 - (1/n) * np.sum(Sigma**2 / (Sigma+nu1)**2)
    Bias = BiasNum / BiasDen

    VarNum =noise**2 * np.sum(Sigma**2 / (Sigma+nu1)**2)
    VarDen = n - np.sum(Sigma**2 / (Sigma+nu1)**2)
    Var = VarNum/VarDen

    return Var, Bias

In [59]:
# Let us define our target function f
def target_function(X):
    # Usiamo una proiezione casuale fissa per definire il target
    w_true = np.random.randn(100) 
    w_true /= np.linalg.norm(w_true)
    # Target = sin(3 * x_proj) + parte quadratica
    proj = X @ w_true
    return np.sin(3 * proj) + 0.1 * (proj**2)

# And now create our KRR problem ... Number of samples:
NSamples = 600
# and their dimension
d = 100

X = np.random.randn(NSamples, d)
noise = 0.1
y = target_function(X) + noise * np.random.randn(NSamples)
# Regularization parameter and gamma parameter for RBF kernel
gamma = 0.5
lambd = 1e-5
# Approximate eigenvalues xi_k and beta decomposition with the empirical diagonalization procedure
Sigma, beta = empiricalDiagonalization(target_function=target_function, gamma = gamma)
Var, Bias = computeDeterministicEquivalent(NSamples, lambd, noise, Sigma, beta)
print(f"Deterministic Equivalent - Variance: {Var:.6f}, Bias: {Bias:.6f}, Total: {Var + Bias:.6f}")

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 1000)

The computation of the deterministic equivalent is supposed to be perfectly deterministic; however, the empirical diagonalization procedure injects stochasticity into there results.

Now given the same Kernel Ridge regression problem, let's compute the empirical risk using SciKitLearn KRR API:

In [55]:
from sklearn.kernel_ridge import KernelRidge
# Build and train the KRR model
krr = KernelRidge(kernel='rbf', alpha=NSamples*lambd, gamma=0.5)
krr.fit(X, y)
# To approximate the true risk, we can use a large test set and approximate via Monte Carlo
Ntest = 10000
X_test = np.random.randn(10000, d)
y_test = target_function(X_test)
y_pred = krr.predict(X_test)    
risk = np.mean((y_test - y_pred)**2)

#test_risk = get_risk_sklearn(krr, X_test, y_test)

print(f"Empirical risk for the KRR model: {risk:.6f}")

Empirical risk for the KRR model: 0.539619


The results seems to be nicely in agreement, suggesting the validity of the kernel limit for $R_{n,p}$. Let's try and evalute the learning curve:

In [58]:
# --- PARAMETRI FISSI ---
d = 100
gamma = 0.01      # 1/d è lo scaling corretto. 0.5 era troppo grande.
lambd = 1e-5
noise = 0.1

# --- 1. CALCOLO SPETTRO OPERATORE (UNA VOLTA SOLA) ---
# Sigma e beta dipendono dalla distribuzione dei dati e dalla funzione target,
# NON da quanti dati usi per il training. Calcolali su una popolazione grande.
# Nota: Assicurati che la tua funzione empiricalDiagonalization usi un N interno grande (es. 2000 o 5000)
Sigma_op, beta_op = empiricalDiagonalization(target_function, gamma=gamma)

# --- 2. LOOP SUI SAMPLE SIZE ---
NValues = np.arange(50, 4001, 200)
EmpiricalRisks = []
DeterministicRisks = []

for N in NValues:
    NSamples = N

    # Generazione Training Set corrente
    X = np.random.randn(NSamples, d)
    y = target_function(X) + noise * np.random.randn(NSamples)
    
    # --- TEORIA (Deterministic Equivalent) ---
    # Usiamo Sigma_op e beta_op calcolati fuori
    Var, Bias = computeDeterministicEquivalent(NSamples, lambd, noise, Sigma_op, beta_op)
    DeterministicRisks.append(Var + Bias)

    # --- SIMULAZIONE (Kernel Ridge Regression) ---
    # IMPORTANTE: Usa la variabile 'gamma', non 0.5 hardcodato!
    krr = KernelRidge(kernel='rbf', alpha=NSamples*lambd, gamma=gamma)
    krr.fit(X, y)
    
    # Test (Monte Carlo approximation of Risk)
    # Usiamo un test set fisso o rigenerato (qui rigenerato va bene)
    X_test = np.random.randn(2000, d) # 2000 è sufficiente per una stima veloce
    y_test = target_function(X_test)
    y_pred = krr.predict(X_test)    
    
    risk = np.mean((y_test - y_pred)**2)
    EmpiricalRisks.append(risk)

# Plotting the results
plt.figure(figsize=(8, 6))
plt.plot(NValues, EmpiricalRisks, label='Empirical Risk (KRR)', marker='o', alpha=0.6)
plt.plot(NValues, DeterministicRisks, label='Deterministic Equivalent', marker='x', linestyle='--')
plt.xlabel('Number of Samples (N)')
plt.ylabel('Risk (MSE)')
plt.title(f'Empirical vs Deterministic (d={d}, $\gamma$={gamma})')
plt.legend()
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.yscale('log') # La scala logaritmica aiuta a vedere meglio la discesa
plt.show()

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 1000)